In [1]:
import codecs
import yaml

import rqalpha as rqa
import rqalpha_plus
import rqdatac as rqd
import pandas as pd
import numpy as np
import scipy.stats as st
import time
from datetime import datetime, timedelta
import utils

In [2]:
def _init_rqdata(conf_path):
    with codecs.open(conf_path, 'r', encoding='utf8') as stream:
        conf = yaml.load(stream)
        if 'proxy' in conf:
            proxy_info = (conf['proxy'].get('type'),
                conf['proxy'].get('host'),
                conf['proxy'].get('port'),
                conf['proxy'].get('user'),
                conf['proxy'].get('password'))
        else:
            proxy_info = None
        rqd.init(conf['rqdata_username'], conf['rqdata_password'],
            (conf['rqdata_host'], conf['rqdata_port']), proxy_info=proxy_info)

_init_rqdata('rqpro.yml')

F:\anaconda\lib\site-packages\ipykernel_launcher.py:3: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  This is separate from the ipykernel package so we can avoid doing imports until
F:\anaconda\lib\site-packages\rqdatac\client.py:190: UserWarning: rqdatac is already inited. Settings will be changed.
  warnings.warn("rqdatac is already inited. Settings will be changed.", stacklevel=0)


In [3]:
def get_target_stocks(context, factor, ascending, percent_selected, min_selected, grouper):
    """获得当天需要买入的股票"""
    # 前一个交易日
    selection_date = rqd.get_previous_trading_date(context.now)
    # 前一个交易日的指数成分股
    universe = rqd.index_components(context.index_stockpool, selection_date)
    # 剔除停牌
    universe = utils.drop_suspended(universe, selection_date)
    # 剔除ST
    universe = utils.drop_st(universe, selection_date)
    # 剔除上市小于60天
    universe = utils.drop_recently_listed(universe, selection_date, 60)
    # 获得因子分数
    scores = utils.get_factor(universe, factor, selection_date, selection_date)
    # 返回需要买入的股票
    return utils.select_top_N_percent(
        universe, selection_date, scores, percent_selected, ascending, min_selected,
        grouper=grouper,
    )


def get_target_portfolio(context):
    # =====返回每只股票的权重Series=====
    date = context.now
    universe = context.target_stocks
    benchmark = context.index_stockpool
    # 今天指数成分股
    components = rqd.index_components(benchmark,date)
    # 股票所属行业series
    industry = utils.get_industry(components,date)
    # 指数行业权重
    industry_weight = rqd.index_weights(benchmark,date).groupby(industry).sum()
    # 目标股票市值
    market_cap = rqd.get_factor(universe,'market_cap',date,date)
    # 股票行业分组组内权重
    industry = industry[universe]
    weight_in_industry = market_cap.groupby(industry).apply(lambda x:x/x.sum())
    # 股票行业目标权重
    target_ids_weight = industry.map(industry_weight)
    # 股票最终权重
    return weight_in_industry*target_ids_weight
    
# def init(context):
#     pass


def handle_bar(context, bar_dict):
    # 如果是False(不是月初),直接返回
    if not _should_rebalance(context):
        return
    print(context.now)
    print("现有持仓数:", len(context.portfolio.positions))
    context.target_stocks = get_target_stocks(context, **context.stock_selection_args)
    context.target_portfolio = get_target_portfolio(context)
    print(f"总权重:{context.target_portfolio.sum()},\
          最大权重{context.target_portfolio.max()},\
          最小权重{context.target_portfolio.min()}")
    rebalance(context, bar_dict)


def _should_rebalance(context):
    """判断今天是否调仓"""
    # 获得上一个交易日
    prev_trading_day = rqd.get_previous_trading_date(context.now)
    # 判断是否是月初, 是月初返回True
    is_month_start = (prev_trading_day.month != context.now.month)
    return is_month_start


def rebalance(context, bar_dict):
    
    positions = context.stock_account.positions
    for order_book_id in positions:
        if order_book_id not in context.target_portfolio:
            rqa.api.order_to(order_book_id, 0)
    
    # 对每个股票计算目标价值和当前价值的差值
    # 差值为正的是买单, 反之为卖单
    capital = context.stock_account.total_value * (1 - context.cash_cushion)
    to_sell, to_buy = {}, {}
    _money_for_one_lot = lambda order_book_id: bar_dict[order_book_id].close * 100
    for order_book_id, weight in context.target_portfolio.items():
        # 股票目标价值
        target_value = capital * weight
        # 目标和现有之差
        gap = target_value - positions[order_book_id].market_value
        # 买卖至少大于1手股票价值
        if abs(gap)<_money_for_one_lot(order_book_id):
            continue
        elif gap > 0:
            to_buy[order_book_id] = gap
        else:
            to_sell[order_book_id] = gap

    # to avoid liquidity issue, sell first, buy second
    for order_book_id, value in to_sell.items():
        rqa.api.order_value(order_book_id, value)
    for order_book_id, value in to_buy.items():
        rqa.api.order_value(order_book_id, value)

# 回测

## 参数

In [25]:
MILLION = 1_000_000
BILLION = 1000 * MILLION
code = '000905.XSHG'
fac = 'operating_profit_growth_ratio_ttm'
asc = True
config = {
    "base": {
        "matching_type": "current_bar",
        "start_date": '2014-03-01',
        "end_date": '2020-07-20',
        "frequency": '1d',
        "accounts": {"stock": 0.1 * BILLION},
    },

    "mod": {
        "sys_analyser": {
            "enabled": True,
            "plot": True,
            "benchmark": code,
        },
    },

    "extra": {
        "log_level": 'error',
        "context_vars": {
            'cash_cushion': 0.005,
            'index_stockpool': code,
            'stock_selection_args': {
                'factor': fac,
                'ascending': asc,
                'percent_selected': 0.1,
                'min_selected': 2,
                'grouper': True
            },
        },
    },
}


In [ ]:
res = rqa.run_func(handle_bar=handle_bar, config=config)

F:\anaconda\lib\site-packages\rqdatac\client.py:190: UserWarning: rqdatac is already inited. Settings will be changed.
  warnings.warn("rqdatac is already inited. Settings will be changed.", stacklevel=0)
[2014-03-03 15:00:00.000000] INFO: user_log: 2014-03-03 15:00:00
[2014-03-03 15:00:00.000000] INFO: user_log: 现有持仓数: 0
[2014-03-03 15:00:00.000000] INFO: user_log: 总权重:0.9998,          最大权重0.050144583115494995,          最小权重0.0010568843671809978
[2014-04-01 15:00:00.000000] INFO: user_log: 2014-04-01 15:00:00
[2014-04-01 15:00:00.000000] INFO: user_log: 现有持仓数: 59
[2014-04-01 15:00:00.000000] INFO: user_log: 总权重:1.0003,          最大权重0.05427462190357307,          最小权重0.001078233202836446
[2014-05-05 15:00:00.000000] INFO: user_log: 2014-05-05 15:00:00
[2014-05-05 15:00:00.000000] INFO: user_log: 现有持仓数: 60
[2014-05-05 15:00:00.000000] INFO: user_log: 总权重:1.0001,          最大权重0.038534355678879965,          最小权重0.0010736469506708094
[2014-06-03 15:00:00.000000] INFO: user_log: 2014-06-03 1

---

In [ ]:
res['sys_analyser']['trades'].to_csv('operating_profit_growth_ratio_ttm.csv')